In [69]:
import numpy as np
import pandas as pd
# %matplotlib widget
# import seaborn as sns
from matplotlib import pyplot as plt


In [70]:
X_train = np.load('./data/X_train.npy')
X_test = np.load('./data/X_test.npy')

y_train = pd.read_csv('./data/y_train.csv')
# y_test = pd.read_csv('./data/y_test.csv')
y_test = pd.read_csv('./data/sample_submission.csv')

print(X_train.shape, y_train.shape)
print(X_test.shape)

(323, 40, 100) (32300, 2)
(54, 40, 100)


In [71]:
print(y_train.shape)
y_train

(32300, 2)


,sample-timestep,class
0,0-0,0
1,0-1,0
2,0-2,0
3,0-3,0
4,0-4,0
...,...,...
32295,322-95,4
32296,322-96,4
32297,322-97,4
32298,322-98,4


In [72]:
y_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32300 entries, 0 to 32299
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   sample-timestep  32300 non-null  object
 1   class            32300 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 504.8+ KB


In [73]:
y_train[['sample_id', 'timestep']] = y_train['sample-timestep'].str.split('-', n=1, expand=True).astype(int)
y_train = y_train.pivot(index='sample_id', columns='timestep', values='class')
y_train_index = y_train.index
y_train = y_train.sort_index() # на всякий случай
y_train = y_train.values

print(y_train.shape)
y_train

(323, 100)


timestep,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
sample_id,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,6,6,6,6,6,6,6,6,6,6
3,6,6,6,6,6,6,6,6,6,6,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,5,5,5,5,5,5,5,5,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318,0,0,0,0,0,0,0,0,0,0,...,8,8,8,8,8,8,8,8,8,8
319,8,8,8,8,8,8,8,8,8,8,...,0,0,0,0,0,0,0,0,0,0
320,0,0,0,0,0,0,0,0,0,0,...,7,7,7,7,7,7,7,7,7,7


In [74]:
y_test[['sample_id', 'timestep']] = y_test['sample-timestep'].str.split('-', n=1, expand=True).astype(int)
y_test = y_test.pivot(index='sample_id', columns='timestep', values='class')
y_test_index = y_test.index
y_test = y_test.sort_index() # на всякий случай
y_test = y_test.values

print(y_test.shape)
y_test

(54, 100)


array([[0, 7, 3, ..., 1, 2, 8],
       [7, 3, 4, ..., 6, 7, 0],
       [0, 0, 2, ..., 3, 1, 5],
       ...,
       [5, 1, 0, ..., 1, 7, 3],
       [4, 0, 5, ..., 4, 7, 1],
       [5, 2, 0, ..., 0, 5, 1]], dtype=int64)

In [45]:
y_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 6, 6, 6],
       ...,
       [0, 0, 0, ..., 7, 7, 7],
       [7, 7, 7, ..., 0, 0, 0],
       [0, 0, 0, ..., 4, 4, 4]], dtype=int64)

In [32]:
df_y_train = []
for idx, arr in zip(y_train_index, y_train):
    
    series = pd.Series(arr)
    series.index = [f"{idx}-{i}" for i in series.index]
    df_y_train.append(series)
    
df_y_train = pd.concat(df_y_train)
df_y_train = df_y_train.reset_index()
df_y_train.columns = ["sample-timestep", "class"]
df_y_train

,sample-timestep,class
0,0-0,0
1,0-1,0
2,0-2,0
3,0-3,0
4,0-4,0
...,...,...
32295,322-95,4
32296,322-96,4
32297,322-97,4
32298,322-98,4


In [33]:
X_train.shape, y_train.shape

((323, 40, 100), (323, 100))

# Baseline

In [82]:
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score

# Преобразование X_train
X_train_reshaped = X_train.reshape(323, 40 * 100)  # Reshape to 2D

# Преобразование y_train
y_train_reshaped = y_train.reshape(323, 100)

# Нормализация данных
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_reshaped)

# Создание и обучение модели
base_model = LogisticRegression(multi_class='ovr', max_iter=1000)
multi_output_model = MultiOutputClassifier(base_model, n_jobs=-1)
multi_output_model.fit(X_train_scaled, y_train_reshaped)

# Для предсказания на тестовых данных
X_test = np.load('data/X_test.npy')
X_test_reshaped = X_test.reshape(-1, 40 * 100)
X_test_scaled = scaler.transform(X_test_reshaped)

# Предсказание
y_pred = multi_output_model.predict(X_test_scaled)

# y_pred уже имеет формат (n_samples, 100), 
# где каждый элемент - предсказанный жест от 0 до 8

In [84]:
y_test_reshaped = y_test.reshape(X_test.shape[0], 100)

In [86]:
y_test_reshaped

array([[0, 7, 3, ..., 1, 2, 8],
       [7, 3, 4, ..., 6, 7, 0],
       [0, 0, 2, ..., 3, 1, 5],
       ...,
       [5, 1, 0, ..., 1, 7, 3],
       [4, 0, 5, ..., 4, 7, 1],
       [5, 2, 0, ..., 0, 5, 1]], dtype=int64)

In [85]:
# Расчет F1-score
# Мы будем использовать micro-average, так как это многоклассовая задача
f1 = f1_score(y_test_reshaped.ravel(), y_pred.ravel(), average='micro')
print(f"Micro-averaged F1-score: {f1}")

# Если вы хотите посмотреть F1-score для каждого класса
f1_per_class = f1_score(y_test_reshaped.ravel(), y_pred.ravel(), average=None)
for i, score in enumerate(f1_per_class):
    print(f"F1-score for class {i}: {score}")

Micro-averaged F1-score: 0.10462962962962963
F1-score for class 0: 0.1702127659574468
F1-score for class 1: 0.07203389830508475
F1-score for class 2: 0.08172043010752687
F1-score for class 3: 0.0654627539503386
F1-score for class 4: 0.07583965330444203
F1-score for class 5: 0.0718294051627385
F1-score for class 6: 0.07667031763417306
F1-score for class 7: 0.06538895152198422
F1-score for class 8: 0.0732519422863485


In [88]:
df_y_test = []
for idx, arr in zip(y_test_index, y_test_reshaped):
    
    series = pd.Series(arr)
    series.index = [f"{idx}-{i}" for i in series.index]
    df_y_test.append(series)
    
df_y_test = pd.concat(df_y_test)
df_y_test = df_y_test.reset_index()
df_y_test.columns = ["sample-timestep", "class"]
df_y_test

,sample-timestep,class
0,323-0,0
1,323-1,7
2,323-2,3
3,323-3,0
4,323-4,5
...,...,...
5395,376-95,3
5396,376-96,1
5397,376-97,0
5398,376-98,5


In [90]:
y_test_file = df_y_test.copy()
y_test_file.to_csv('./y_test_predicted.csv', index=False)
y_test_file

,sample-timestep,class
0,323-0,0
1,323-1,7
2,323-2,3
3,323-3,0
4,323-4,5
...,...,...
5395,376-95,3
5396,376-96,1
5397,376-97,0
5398,376-98,5
